In [6]:
import mlflow
#Set up the MLflow tracking server
mlflow.set_tracking_uri('http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/')

In [7]:
# Set or create an experiment
mlflow.set_experiment("Exp3: BoW vs TfIdf")

<Experiment: artifact_location='s3://youtubecommentmlflow/720032388652741727', creation_time=1741955876973, experiment_id='720032388652741727', last_update_time=1741955876973, lifecycle_stage='active', name='Exp3: BoW vs TfIdf', tags={}>

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [ ]:
df = pd.read_csv('C:/test/youtube_comment_sentiment_analysis/Artifacts/youtubecomment_preprocessing.csv')
df.shape

(15833, 2)

In [ ]:
# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    

    X_train, X_test, y_train, y_test = train_test_split(df['Comment'], df['Sentiment'], test_size=0.2, random_state=42, stratify=df['Sentiment'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    smote = SMOTE(random_state=42)
    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("C:/test/youtube_comment_sentiment_analysis/Artifacts//confusion_matrix2.png")
        mlflow.log_artifact("C:/test/youtube_comment_sentiment_analysis/Artifacts//confusion_matrix2.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2025/03/14 18:09:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpoarsy22u\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 18:09:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727/runs/848a997137fb4fb19b792febbf4dca56
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727


2025/03/14 18:10:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpa_20r_1w\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 18:10:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727/runs/f5f1f4d194ea4fea8e50a9427e39f76b
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727


2025/03/14 18:10:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmphetnq24s\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 18:10:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727/runs/67a486604943496a913d6d1d748e8d6f
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727


2025/03/14 18:12:36 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpvr3nob1n\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 18:12:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727/runs/67c29a2521ea4841befca86bb43d3d37
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727


2025/03/14 18:13:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\PARTHI~1\AppData\Local\Temp\tmpne4boxbm\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.0.2', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/03/14 18:13:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727/runs/5fdee546f8a34834a24e805105e0f620
🧪 View experiment at: http://ec2-54-173-92-7.compute-1.amazonaws.com:5000/#/experiments/720032388652741727
